In [8]:
import boto3
import json
import pandas as pd
import os


import src.config as con

In [9]:
# set up AWS credentials
aws_access_key_id = con.aws_access_key_id
aws_secret_access_key = con.aws_secret_access_key
region_name = con.region_name

# specify bucket name and JSON file key
bucket_name = 'ec2bucket-test'

In [10]:
json_file_key = 'data/song_data/TRAAAAW128F429D538.json'

In [11]:
# set up S3 client
s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, 
                  aws_secret_access_key=aws_secret_access_key,
                  region_name=region_name)

In [12]:
# read the JSON file from S3
response = s3.get_object(Bucket=bucket_name, Key=json_file_key)
json_data = response['Body'].read().decode('utf-8')

In [13]:
json_data

'{"num_songs": 1, "artist_id": "ARD7TVE1187B99BFB1", "artist_latitude": null, "artist_longitude": null, "artist_location": "California - LA", "artist_name": "Casual", "song_id": "SOMZWCG12A8C13C480", "title": "I Didn\'t Mean To", "duration": 218.93179, "year": 0}'

In [16]:
# Parse the JSON object into a dictionary
data_dict = json.loads(json_data)

# Convert the dictionary into a pandas DataFrame
df = pd.DataFrame([data_dict])

# Display the resulting DataFrame
df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,None,None,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


In [40]:
# Set bucket and folder names
bucket_name = 'ec2bucket-test'
folder_name = 'data/song_data/'

In [43]:
# list all objects under the prefix in the bucket
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_name)

# extract the object keys
object_keys = [obj['Key'] for obj in response['Contents']]

# loop through the object keys and load each JSON file into a DataFrame
dfs = []
for object_key in object_keys:
    response = s3.get_object(Bucket=bucket_name, Key=object_key)
    json_data = response['Body'].read().decode('utf-8')
    data_dict = json.loads(json_data)
    df = pd.DataFrame([data_dict])
    dfs.append(df)

# concatenate all DataFrames into a single one
df = pd.concat(dfs, ignore_index=True)

In [44]:
result_df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0
1,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969
2,1,ARKRRTF1187B9984DA,NaN,NaN,,Sonora Santanera,SOXVLOJ12AB0189215,Amor De Cabaret,177.47546,0
3,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
4,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
...,...,...,...,...,...,...,...,...,...,...
66,1,AR8IEZO1187B99055E,NaN,NaN,,Marc Shaiman,SOINLJW12A8C13314C,City Slickers,149.86404,2008
67,1,AR558FS1187FB45658,NaN,NaN,,40 Grit,SOGDBUF12A8C140FAA,Intro,75.67628,2003
68,1,ARVBRGZ1187FB4675A,NaN,NaN,,Gwen Stefani,SORRZGD12A6310DBC3,Harajuku Girls,290.55955,2004
69,1,ARWB3G61187FB49404,NaN,NaN,"Hamilton, Ohio",Steve Morse,SODAUVL12A8C13D184,Prognosis,363.85914,2000


In [45]:
song_df = df.copy()

In [46]:
song_df.columns

Index(['num_songs', 'artist_id', 'artist_latitude', 'artist_longitude',
       'artist_location', 'artist_name', 'song_id', 'title', 'duration',
       'year'],
      dtype='object')

In [47]:
song_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   num_songs         71 non-null     int64  
 1   artist_id         71 non-null     object 
 2   artist_latitude   31 non-null     float64
 3   artist_longitude  31 non-null     float64
 4   artist_location   71 non-null     object 
 5   artist_name       71 non-null     object 
 6   song_id           71 non-null     object 
 7   title             71 non-null     object 
 8   duration          71 non-null     float64
 9   year              71 non-null     int64  
dtypes: float64(3), int64(2), object(5)
memory usage: 5.7+ KB


In [49]:
song_df.describe()

,num_songs,artist_latitude,artist_longitude,duration,year
count,71.0,31.000000,31.000000,71.000000,71.000000
mean,1.0,36.552972,-73.251233,239.729676,785.957746
std,0.0,12.431023,36.058076,106.562779,980.957119
min,1.0,-13.442000,-122.420050,29.544040,0.000000
25%,1.0,34.730385,-92.039350,170.722810,0.000000
50%,1.0,38.899100,-80.112780,228.597100,0.000000
75%,1.0,40.910475,-73.986780,286.366890,1992.500000
max,1.0,56.276090,15.967600,599.248530,2008.000000
